In [2]:
import numpy as np
import urllib
import os
import logging
from pathlib import Path
import pandas as pd

In [55]:
# Load in csvs
WHR_file_name = 'world-happiness-report.csv'
WHR2021_file_name = 'world-happiness-report-2021.csv'
CPDS_file_name = 'CPDS_1960-2019_Update_2021.xlsx'
WHR_df = pd.read_csv(f'data\{WHR_file_name}')
WHR2021_df = pd.read_csv(f'data\{WHR2021_file_name}')
CPDS_df = pd.read_excel(f'data\{CPDS_file_name}')


# Clean Data
WHR_df = WHR_df.drop(columns = ['Positive affect','Negative affect'])
WHR2021_df['year'] = 2021
WHR2021_df = WHR2021_df.drop(columns = ['Standard error of ladder score','upperwhisker','lowerwhisker','Ladder score in Dystopia',
                          'Explained by: Log GDP per capita','Explained by: Social support','Explained by: Healthy life expectancy',
                          'Explained by: Freedom to make life choices','Explained by: Generosity','Explained by: Perceptions of corruption',
                          'Dystopia + residual'])
WHR2021_df = WHR2021_df.rename(columns = {'Ladder score':'Life Ladder','Logged GDP per capita':'Log GDP per capita',
                            'Healthy life expectancy':'Healthy life expectancy at birth'})

dic = {}
for i in WHR2021_df['Regional indicator'].unique():
    dic[i]=list(WHR2021_df[WHR2021_df['Regional indicator'] ==i].groupby('Country name').size().index)
dic['Sub-Saharan Africa'].append('Angola')
dic['Latin America and Caribbean'].append('Belize')
dic['South Asia'].append('Bhutan')
dic['Sub-Saharan Africa'].append('Central African Republic')
dic['Sub-Saharan Africa'].append('Congo (Kinshasa)')
dic['Latin America and Caribbean'].append('Cuba')
dic['Sub-Saharan Africa'].append('Djibouti')
dic['Latin America and Caribbean'].append('Guyana')
dic['Middle East and North Africa'].append('Oman')
dic['Middle East and North Africa'].append('Qatar')
dic['Sub-Saharan Africa'].append('Somalia')
dic['Sub-Saharan Africa'].append('Somaliland region')
dic['Sub-Saharan Africa'].append('South Sudan')
dic['Middle East and North Africa'].append('Sudan')
dic['Latin America and Caribbean'].append('Suriname')
dic['Middle East and North Africa'].append('Syria')
dic['Latin America and Caribbean'].append('Trinidad and Tobago')

def find_region(x):
    '''Helper Function to Return Region Name based on the Country Name entered'''
    for reg in dic.keys():
        for c in dic[reg]:
            if x == c:
                return reg
WHR_df['Regional indicator'] = WHR_df['Country name'].apply(find_region)

cmbd_WHR_df = pd.concat([WHR_df,WHR2021_df])

CPDS_file_name = 'CPDS_1960-2019_Update_2021.xlsx'
CPDS_df = pd.read_excel(f'data\{CPDS_file_name}')
CPDS_columns_to_drop = ['year_01','country_01','elect','year_02','country_02']

CPDS_df = CPDS_df.drop(columns = CPDS_columns_to_drop)
CPDS_df = CPDS_df.dropna(subset = ['year','country'])
def convert_to_int(x):
    return int(x)

CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)

all_data = pd.merge(cmbd_WHR_df,CPDS_df,how = 'left',left_on = ['year','Country name'],right_on = ['year','country'])

CPDS_df

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10249.099609,6353.600098,849.000000,8.283654,1960.0,Australia
1,1961,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10452.099609,6486.899902,868.099976,8.305508,1961.0,Australia
2,1962,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10649.500000,6631.100098,886.500000,8.324334,1962.0,Australia
3,1963,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10851.200195,6771.700195,905.400024,8.343778,1963.0,Australia
4,1964,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,11071.299805,6919.500000,920.000000,8.309773,1964.0,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,2015,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,6.54,...,0.25,47.6,43.7,37.6,320918.000000,212075.203125,47667.699219,14.853545,2015.0,USA
1754,2016,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,5.88,...,0.25,47.1,43.0,37.3,323186.000000,212871.906250,49224.000000,15.230858,2016.0,USA
1755,2017,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,81.89,...,0.25,47.1,42.8,37.3,325220.000000,213390.500000,50793.898438,15.618320,2017.0,USA
1756,2018,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,92.73,...,0.25,47.1,42.7,37.3,326949.000000,213850.796875,52431.199219,16.036507,2018.0,USA


,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Regional indicator,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,Afghanistan,2008,3.724,7.370,0.451,50.800,0.718,0.168,0.882,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009,4.402,7.540,0.552,51.200,0.679,0.190,0.850,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010,4.758,7.647,0.539,51.600,0.600,0.121,0.707,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011,3.832,7.620,0.521,51.920,0.496,0.162,0.731,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012,3.783,7.705,0.521,52.240,0.531,0.236,0.776,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,Lesotho,2021,3.512,7.926,0.787,48.700,0.715,-0.131,0.915,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,Botswana,2021,3.467,9.782,0.784,59.269,0.824,-0.246,0.801,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,Rwanda,2021,3.415,7.676,0.552,61.400,0.897,0.061,0.167,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,Zimbabwe,2021,3.145,7.943,0.750,56.201,0.677,-0.047,0.821,Sub-Saharan Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
all_data.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Regional indicator,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,South Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
temp_df = all_data.drop(columns = ['Regional indicator'])

In [59]:

columns = temp_df.columns[2:]
new_col_names = []
for i in range(2006,2022):
    year = str(i)
    for col in columns:
        col_name = f'{col}_{year}'
        new_col_names.append(col_name)

groups = temp_df.groupby('Country name')

num_cols = len(columns)
missing_year_data = [np.nan for i in range(num_cols)]
data = {}
for key, group in groups:
    row_data = []
    group_years = list(group['year'])
    c_group = group.drop(columns = 'Country name')
    c_group = c_group.set_index('year')
    for i in range(2006,2022):
        if i in group_years:
            row_data += list(c_group.loc[i])
        else:
            row_data += missing_year_data
    data[key] = row_data
    

reformatted_data = pd.DataFrame.from_dict(data, orient='index',columns = new_col_names)
reformatted_data

,Life Ladder_2006,Log GDP per capita_2006,Social support_2006,Healthy life expectancy at birth_2006,Freedom to make life choices_2006,Generosity_2006,Perceptions of corruption_2006,country_2006,countryn_2006,iso_2006,...,emprot_temp_2021,prefisc_gini_2021,pretran_gini_2021,postfisc_gini_2021,pop_2021,pop15_64_2021,pop65_2021,elderly_2021,year_13_2021,country_13_2021
Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,6.313,9.942,0.938,66.82,0.733,-0.157,0.852,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,6.525,9.460,0.946,65.46,0.798,-0.031,0.646,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vietnam,5.294,8.335,0.888,65.86,0.886,0.015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zambia,4.824,7.817,0.798,44.26,0.721,-0.006,0.785,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
reformatted_data.to_csv('final_data/WHD_and_CPDS.csv')

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960.0,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10249.099609,6353.600098,849.000000,8.283654,1960.0,Australia
1,1961.0,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10452.099609,6486.899902,868.099976,8.305508,1961.0,Australia
2,1962.0,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10649.500000,6631.100098,886.500000,8.324334,1962.0,Australia
3,1963.0,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,10851.200195,6771.700195,905.400024,8.343778,1963.0,Australia
4,1964.0,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.00,...,NaN,NaN,NaN,NaN,11071.299805,6919.500000,920.000000,8.309773,1964.0,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,2015.0,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,6.54,...,0.25,47.6,43.7,37.6,320918.000000,212075.203125,47667.699219,14.853545,2015.0,USA
1754,2016.0,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,5.88,...,0.25,47.1,43.0,37.3,323186.000000,212871.906250,49224.000000,15.230858,2016.0,USA
1755,2017.0,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,81.89,...,0.25,47.1,42.8,37.3,325220.000000,213390.500000,50793.898438,15.618320,2017.0,USA
1756,2018.0,USA,36.0,USA,840.0,1.0,0.0,0.0,0.0,92.73,...,0.25,47.1,42.7,37.3,326949.000000,213850.796875,52431.199219,16.036507,2018.0,USA


In [43]:
def convert_to_int(x):
    return int(x)

CPDS_df['year'] = CPDS_df[['year']].applymap(convert_to_int)
CPDS_df.head()

,year,country,countryn,iso,iso3n,cpds1,poco,eu,emu,gov_right1,...,emprot_temp,prefisc_gini,pretran_gini,postfisc_gini,pop,pop15_64,pop65,elderly,year_13,country_13
0,1960,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10249.099609,6353.600098,849.000000,8.283654,1960.0,Australia
1,1961,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10452.099609,6486.899902,868.099976,8.305508,1961.0,Australia
2,1962,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10649.500000,6631.100098,886.500000,8.324334,1962.0,Australia
3,1963,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,10851.200195,6771.700195,905.400024,8.343778,1963.0,Australia
4,1964,Australia,1.0,AUS,36.0,1.0,0.0,0.0,0.0,100.0,...,NaN,NaN,NaN,NaN,11071.299805,6919.500000,920.000000,8.309773,1964.0,Australia
